In [1]:
import json
import random
from glob import glob
from tqdm import tqdm
from transformers import AutoTokenizer

/home/husein/.local/lib/python3.8/site-packages/requests/__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.15) or chardet (5.2.0)/charset_normalizer (2.0.7) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [2]:
synthetic = []

for f in glob('/home/husein/ssd3/instructions/synthetic-*.jsonl'):
    with open(f) as fopen:
        for l in fopen:
            
            if '```' in l:
                continue
                
            l = json.loads(l)
            
            if l['rejected_ins'] or l['rejected_output']:
                continue
                
            if l['instruction_ms'] is None:
                continue
            
            if l['output_ms'] is None:
                continue
                
            synthetic.append({
                'prompt_input': None,
                'input': l['instruction_ms'].strip(),
                'output': l['output_ms'].strip(),
            })
                
len(synthetic)

252244

In [3]:
general_qa = []
for f in glob('/home/husein/ssd3/chatgpt4-malaysian-general-qa/*.jsonl'):
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            q = l.get('question_ms', l['question']).strip()
            if not len(q):
                continue
            a = l.get('answer_ms', l['answer'])
            if a is None:
                continue
            a = a.strip()
            if not len(a):
                continue
                
            d = {
                'prompt_input': None,
                'input': q,
                'output': a,
            }
            general_qa.append(d)
            
len(general_qa)

70258

In [4]:
skip = [
    'always better to find peaceful',
    'non-violent ways to express',
    'can lead to severe consequences',
    'still have questions or concerns about',
    'that doing so is illegal',
    'to report the incident to',
    'would recommend consulting with',
    'indonesian',
    'translates to',
    'idiom in',
    'in English',
    'in Malay',
    # 'language model'
]

rejected = [
    'math', 
    'multiturn', 
    'dangerous', 
    'starter', 
    'conversation', 
    'rag', 
    'audio',
    'ahli-parlimen',
    'disagree'
]

mixtral_instructions = []
files = glob('/home/husein/ssd3/soalan-augmentation/mixtral*.jsonl')
files = [f for f in files if all([r not in f for r in rejected])]
for f in files:
    with open(f) as fopen:
        for l in fopen:
            l_lower = l.lower()
            l = json.loads(l)
            if any([s in l_lower for s in skip]):
                # print(l)
                continue
            
            q = l.get('question', '').strip()
            if not len(q):
                continue
            if not q.endswith('?'):
                continue
            if len(q) < 200 and '?' not in q:
                # print(q)
                continue
            a = l.get('answer_ms')
            if a is None:
                continue
            a = a.strip()
            if not len(a):
                continue
            if len(a) < len(q):
                continue
            if len(a) < 100:
                continue
                
            d = {
                'prompt_input': None,
                'input': q,
                'output': a,
            }
            mixtral_instructions.append(d)
            
files = glob('/home/husein/ssd3/soalan-augmentation/*math*.jsonl')
files = [f for f in files if 'conversation' not in f]
for f in files:
    with open(f) as fopen:
        for l in fopen:
            l = json.loads(l)
            q = l.get('question', '').strip()
            if not len(q):
                continue
            a = l.get('answer_ms')
            if a is None:
                continue
            a = a.strip()
            if not len(a):
                continue
            if len(a) < len(q):
                continue
            if len(a) < 100:
                continue
                
            d = {
                'prompt_input': None,
                'input': q,
                'output': a,
            }
            mixtral_instructions.append(d)

len(mixtral_instructions)

265871

In [5]:
random_instructions = {
    0: synthetic,
    1: general_qa,
    2: mixtral_instructions
}

In [6]:
roles = {
    'user': '<manusia>',
    'assistant': '<bot>'
}

In [7]:
random_combine = []
for i in tqdm(range(80000)):
    combine = []
    for _ in range(random.randint(2, 4)):
        index = random.randint(0, len(random_instructions) - 1)
        choice = random.choice(random_instructions[index])
        s = f"<manusia>: {choice['input']}"
        combine.append(s)
        s = f"<bot>: {choice['output']}"
        combine.append(s)
    data = '\n'.join(combine).strip()
    a = {
        'prompt_input': None,
        'input': data,
        'output': None
    }
    random_combine.append(a)

100%|██████████████████████████████████████████████████████████████████████████████████████████| 80000/80000 [00:00<00:00, 130343.35it/s]


In [8]:
break_at = [
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI',
    'model bahasa AI',
    'mempunyai kepercayaan atau pendapat peribadi',
    'tidak mempunyai pendapat peribadi',
    'tidak mempunyai kepercayaan',
    'tidak mempunyai falsafah peribadi',
    'tidak mempunyai pengalaman peribadi',
    'tidak mempunyai pendapat atau pengalaman peribadi',
    'tidak mempunyai maklumat terkini',
    'tidak mempunyai emosi peribadi',
    'tidak mempunyai keutamaan',
    'saya tidak mempunyai akses',
    'tidak mempunyai pengalaman',
    'saya tidak mempunyai keupayaan',
    'tidak mempunyai keupayaan',
    'tidak mempunyai hubungan',
    'tidak mempunyai maklumat',
    'Saya tidak mempunyai',
    'Saya tidak pernah',
    'help.openai.com',
    'openai',
    'cannot have personal opinions',
    's an ai language model',
    "i'm sorry",
    'many factors',
    'lgbt',
    'lesbian',
    'gender-neutral',
    'remain neutral',
    'without bias',
    'and neutral',
    'more inclusive',
    'neutrality',
    'non-bias',
    'discrimination',
    'avoid any forms of discrimination',
    'regardless of their gender',
    'inclusive and tolerant environment',
    'have personal views',
    'sexual orientation should be a top priority',
    's an objective ai',
    'avoid any forms of prejudice or hate',
    'regardless of their personal',
    'you understand this direction',
    'tolerant environment within ai',
    'cannot express my',
    'requires more context',
    'personal opinion',
    'have updated information',
    "don't have personal experiences",
    'there is no information',
    'tidak mempunyai akses kepada data atau maklumat',
    '10 april 2021',
    'ebagai model bahasa AI',
    'ebagai model bahasa ai',
    'model bahasa AI',
    'model bahasa ai',
    'bahasa ai',
    'ebagai model bahasa'
    'hat makes sense',
    'have access to data or information',
    'have access to the data or information',
    'hanya mempunyai akses kepada maklumat umum',
    'hanya boleh memberikan maklumat umum',
    'have personal preferences',
    'not have personal experiences',
    'not capable of having subjective opinions',
    'indonesian'
    'mistral'
]

break_at = list(set(break_at))

In [9]:
from tqdm import tqdm

roles = {
    'user': '<manusia>',
    'assistant': '<bot>'
}

data_instructions_mixtral = []
files = glob('/home/husein/ssd3/soalan-augmentation/*conversation*.jsonl')
for f in files:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            inputs = []
            for no, r in enumerate(l):
                if no < (len(l) - 1):
                    if l[no + 1]['role'] == 'user':
                        next_text = l[no + 1].get('content_ms') or l[no + 1].get('content') or ''
                    else:
                        next_text = l[no + 1].get('content_ms') or ''
                else:
                    next_text = ''
                
                if r['role'] == 'user':
                    current_text = r.get('content_ms') or r.get('content') or ''
                else:
                    current_text = r.get('content_ms') or ''
                    
                if l[no - 1]['role'] == 'user':
                    previous_text = l[no - 1].get('content_ms') or l[no - 1].get('content') or ''
                else:
                    previous_text = l[no - 1].get('content_ms') or ''
                
                # bad pairs
                if r['role'] == 'user' and (len(current_text) < 2 or len(next_text) < 2):
                    # print('a', l, current_text, next_text, '\n')
                    continue
                if r['role'] == 'assistant' and (len(current_text) < 2 or len(previous_text) < 2):
                    continue
                
                # bad pairs
                if r['role'] == 'user' and current_text[:20].lower() == next_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), next_text[:20].lower())
                    continue
                if r['role'] == 'assistant' and current_text[:20].lower() == previous_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), previous_text[:20].lower())
                    continue
                    
                # remove alignments    
                if r['role'] == 'user' and (any([b in current_text.lower() for b in break_at]) or any([b in next_text.lower() for b in break_at])):
                    # print(current_text, next_text)
                    break
                if r['role'] == 'assistant' and (any([b in current_text.lower() for b in break_at]) or any([b in previous_text.lower() for b in break_at])):
                    # print(current_text, next_text)
                    break

                role = roles[r['role']]
                
                s = f"{role}: {current_text}"
                    
                inputs.append(s)

            if len(inputs) % 2 != 0:
                inputs = inputs[:-1]
                
            if not len(inputs):
                continue
            
            if random.random() > 0.4:
                continue
            
            index = random.randint(0, len(random_instructions) - 1)
            choice = random.choice(random_instructions[index])
            manusia = f"<manusia>: {choice['input']}"
            bot = f"<bot>: {choice['output']}"
            
            if random.random() > 0.5:
                inputs.extend([manusia, bot])
            else:
                inputs = [manusia, bot] + inputs

            data = '\n'.join(inputs).strip()
            
            if not len(data):
                continue
                
            a = {
                'prompt_input': None,
                'input': data,
                'output': None
            }
            data_instructions_mixtral.append(a)
            
len(data_instructions_mixtral)

48163it [00:09, 5054.59it/s]
60384it [00:12, 4736.89it/s]
57798it [00:11, 4928.99it/s]


65831

In [10]:
rags = [
    '/home/husein/ssd3/soalan-augmentation/mixtral-rag-question-disagree.jsonl',
    '/home/husein/ssd3/soalan-augmentation/mixtral-audio-instruction.jsonl',
]

roles = {
    'user': '<manusia>',
    'assistant': '<bot>'
}

instructions = []
for f in rags:
    with open(f) as fopen:
        for l in tqdm(fopen):
            l = json.loads(l)
            context = l['context']
            l = l['chat']
            inputs = []
            for no, r in enumerate(l):
                if no < (len(l) - 1):
                    if l[no + 1]['role'] == 'user':
                        next_text = l[no + 1].get('content_ms') or l[no + 1].get('content') or ''
                    else:
                        next_text = l[no + 1].get('content_ms') or ''
                else:
                    next_text = ''
                
                if r['role'] == 'user':
                    current_text = r.get('content_ms') or r.get('content') or ''
                else:
                    current_text = r.get('content_ms') or ''
                    
                if l[no - 1]['role'] == 'user':
                    previous_text = l[no - 1].get('content_ms') or l[no - 1].get('content') or ''
                else:
                    previous_text = l[no - 1].get('content_ms') or ''
                
                # bad pairs
                if r['role'] == 'user' and (len(current_text) < 2 or len(next_text) < 2):
                    # print('a', l, current_text, next_text, '\n')
                    continue
                if r['role'] == 'assistant' and (len(current_text) < 2 or len(previous_text) < 2):
                    continue
                
                # bad pairs
                if r['role'] == 'user' and current_text[:20].lower() == next_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), next_text[:20].lower())
                    continue
                if r['role'] == 'assistant' and current_text[:20].lower() == previous_text[:20].lower():
                    # print(no, r, current_text[:20].lower(), previous_text[:20].lower())
                    continue
                    
                # remove alignments    
                if r['role'] == 'user' and (any([b in current_text.lower() for b in break_at]) or any([b in next_text.lower() for b in break_at])):
                    # print(current_text, next_text)
                    break
                if r['role'] == 'assistant' and (any([b in current_text.lower() for b in break_at]) or any([b in previous_text.lower() for b in break_at])):
                    # print(current_text, next_text)
                    break

                role = roles[r['role']]
                
                if no == 0:
                    current_text = f"{context}\n{current_text}"
                
                s = f"{role}: {current_text}"
                    
                inputs.append(s)

            if len(inputs) % 2 != 0:
                inputs = inputs[:-1]
                
            if not len(inputs):
                continue
                
            if len(inputs) < 3:
                continue
                
            if random.random() > 0.2:
                continue
                
            index = random.randint(0, len(random_instructions) - 1)
            choice = random.choice(random_instructions[index])
            
            manusia = f"<manusia>: {choice['input']}"
            bot = f"<bot>: {choice['output']}"
            
            if random.random() > 0.5:
                inputs.extend([manusia, bot])
            else:
                inputs = [manusia, bot] + inputs

            data = '\n'.join(inputs).strip()
            if not len(data):
                continue
                
            d = {
                'prompt_input': None,
                'input': data,
                'output': None,
            }
            instructions.append(d)

339627it [01:41, 3330.01it/s]
293752it [01:40, 2932.97it/s]


In [11]:
combine_all = data_instructions_mixtral + instructions + random_combine
len(combine_all)

253362

In [12]:
len(combine_all)

253362

In [14]:
def generate_and_tokenize_prompt(row):
    texts = ['<s>']

    if 'function_call' in row:
        t = row['function_call']
        texts.append(f'\n[FUNCTIONCALL]\n{t}\n')

    if '<bot>:' in row['input'] and row['output'] is None:
        inputs, outputs = [], []
        splitted = row['input'].split('<bot>:')
        for i in range(len(splitted) - 1):
            if i == 0:
                human = splitted[i].replace('<manusia>:', '')
            else:
                try:
                    human = splitted[i].split('<manusia>:')[1]
                except:
                    continue
            bot = splitted[i + 1].split('<manusia>:')[0]
            inputs.append(human.strip())
            outputs.append(bot.strip())
    else:
        inputs = [row['input']]
        outputs = [row['output']]

    for u, a in zip(inputs, outputs):
        texts.append(f'[INST] {u.strip()} [/INST] {a.strip()}</s> ')

    prompt = ''.join(texts)
    return {'text': prompt}

In [15]:
from tqdm import tqdm

for i in tqdm(range(len(combine_all))):
    generate_and_tokenize_prompt(combine_all[i])

100%|████████████████████████████████████████████████████████████████████████████████████████| 253362/253362 [00:02<00:00, 106367.58it/s]


In [16]:
with open('prepared-combine-multiple-chats-nocode.jsonl', 'w') as fopen:
    for l in combine_all:
        fopen.write(f'{json.dumps(l)}\n')

In [17]:
!wc -l prepared-combine-multiple-chats-nocode.jsonl

253362 prepared-combine-multiple-chats-nocode.jsonl


In [18]:
!ls -lh prepared-combine-multiple-chats-nocode.jsonl

-rw-r--r-- 1 husein husein 1.3G Jan  14 16:42 prepared-combine-multiple-chats-nocode.jsonl


In [22]:
instructions[0]

{'prompt_input': None,
 'input': '<manusia>: Belanda menjadi kuasa besar dunia pada kurun ke-18 kerana kekuatan maritimnya. Dengan kekuatan ini mereka mempunyai koloni di Perak dan di Indonesia. Tidak banyak kapal dari kurun ini masih wujud. Salah satunya Amsterdam karam di England pada tahun 1749.\n\nBangkainya telah ditemui dan kini sebuah muzium bawah laut akan dibina di Amsterdam untuk memastikan ia dapat dinikmati oleh generasi akan datang. Muzium Docking the Amsterdam ini unik kerana bangkai Amsterdam akan kekal di bawah permukaan laut untuk memastikan ia dapat dikekalkan. Pengunjung akan dapat melihat bahagian tubuh kapal di dalam galeri kaca seperti sebuah akuarium. Dari bahagian atas pengunjung boleh melihat ke bawah kapal yang tenggelam.\n\nAmsterdam ialah sebuah kapal kargo milik Syarikat Hindia Timur Belanda yang dalam perjalanan dari Texel ke Batavia (kini Jakarta) pada tahun 1749. Ia kemudiannya menghadapi masalah ketika berlayar di  Selat Inggeris kerana cuaca yang buruk